In [ ]:
import json #Javascript Object 
import pandas as pd
import requests
import seaborn as sns

In [ ]:
# We doen een "aanvraag" bij de API. Deze url werkt alleen als de rest server gestart is. 
response = requests.get('http://localhost:8080/medish_centrum_randstad/api/netlify?page=1')

In [ ]:
# Checken wat voor data/type we binnen hebben gekregen
response.headers['content-type']

In [ ]:
# Van de teruggekregen data maken we een pandas dataframe
df = pd.read_json(response.text)
df.head()

In [ ]:
# wat we hier willen:

    # De data is opgeslagen in de 'data' kolom. De vorm waarin dit is opgeslagen is een dictionary (voor elke rij)
    # waarvan de keys de variabelen zijn, en de waardes de waardes voor elke 'patient'
    
    # Nu willen we dus elk dictionary verwerken tot een rij in het dataframe, zodat we een rij hebben voor elke patient
    

# Eerst heb ik alle dictionaries in een lijst gezet, om ze als het ware uit het "oude" dataframe te halen:

data_list = df.data.values.tolist()
    
# Hier kunnen we inspecteren hoe het eerste item, i.e., de eerste rij, eruit ziet:
print(data_list[0])
    

In [ ]:
# Nu is het zaak om de dictionaries uit de gemaakte lijst allemaal samen te voegen in een datatframe

df_final = pd.DataFrame.from_dict(data_list)

# We kunnen ons nieuwe dataframe even bekijken:

print(df_final.head())
print()
print(df_final.tail())


In [ ]:
# Wat opvalt is dat het dataframe lijkt te zijn geordend op de variabel 'genetic', 
# bij het eerst x aantal rijen zien we dit terug:

df_final.head(30)

In [ ]:
# Om eventuele problemen met training/test subsets van de data te voorkomen, heb ik ervoor gekozen om de rijen
# in willekeurige volgorde te zetten:

df_final = df_final.sample(frac=1)

In [ ]:
# Checken hoe het er nu uitziet:

print(df_final.head())
print()
print(df_final.tail())

# De rijen zijn nu inderdaad in willekeurige volgorde gezet.

In [ ]:
# We hebben nu een werkbaar dataframe met al onze variabelen in de kolommen, en alle individuele patienten/datapunten
# in de rijen. We kunnen deze nu evt. als een nieuwe csv file exporteren, zonder index:

df_final.to_csv('data_clean.csv', index=False, sep=';')

In [ ]:
# Als een van de eerste stappen in de IDE kunnen we kijken naar wat voor soort variabelen we hebben,
# van welk type deze variabelen zijn (categorisch of numeriek), en ook of er bijvoorbeeld waardes missen. 

# Eerst kijken we naar de datatypen van elke kolom
print(df_final.dtypes)
print(df_final.info())

In [ ]:
# We zijn dat we enkel met getallen (floats en ints) te maken hebben. Nu is de vraag: zijn dit allemaal numerieke waardes
# op een schaal, of zijn er binnen de variabelen misschien categoriën?

print(df_final.nunique())

In [ ]:
# De variabelen 'length', 'mass', 'smoking' en 'lifespan' hebben de meeste verschillende waardes. 'Sugar' en 'exercise'
# lijken allebei te bestaan uit 5 verschillende waardes, variërend van 1 tot 5. Hier gaan we er even vanuit dat er geen 
# waardes tussen de hele getallen liggen, maar dit is wel iets om rekening mee te houden natuurlijk. 

In [ ]:
# Voor de zekerheid bekijken we toch even of alle variabelen enkel hele getallen bevatten:

for c in df_final:
    print((df_final[c].fillna(-9999) % 1  == 0).all())

In [ ]:
# Aangezien alle variabelen dus hele getallen zijn kunnen we alles omzetten naar integers, voor de eenvoud:

for c in df_final:
    df_final[c] = df_final[c].fillna(0.0).astype(int)
    
print(df_final.dtypes)

In [ ]:
# Een volgende stap is om eens te kijken naar wat van de variabelen, hoe zit het bijvoorbeeld met de gemiddelde
# levensverwachting en suikerinname? 

df_final[['sugar', 'lifespan']].groupby(['sugar'], as_index=False).mean().sort_values(by='lifespan', ascending=False)

In [ ]:
# Hier zien we een duidelijke afname in levensverwachting naarmate 'sugar' een hogere waarde aanneemt. 
# Hetzelfde kunnen we doen voor 'exercise'

df_final[['exercise', 'lifespan']].groupby(['exercise'], as_index=False).mean().sort_values(by='lifespan')

In [ ]:
# Wat niet zozeer interessant is voor ons toekomstige model, maar wel interessant: hoe is het gesteld met het bewegen
# in deze dataset; 

In [ ]:
df_final['exercise'].value_counts()

In [ ]:
df_final.loc[df_final['exercise'] == 4]['sugar'].mean()

In [ ]:
#We kunnen nu kijken naar eventuele onderlinge correlaties:
print(df_final.corr())
sns.heatmap(df_final.corr())

In [ ]:
df_final.corr()['lifespan']